# Análisis asignación semanas de mantenimiento


In [9]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from datetime import datetime,timedelta
from utils import (
    extra_week_indicator,
    main_day_weeker,
    gauss_easter
)

weeks_expected_per_year = 365 // 7
fractions_quantity = 8

### Tipos de fechas festivas

_Cálculo de fechas y estrategia para calcular semana en que se fetejan._

**CASO 1:** _Deterministas Regulares_

Usando directamente el `datetime` correspondiente a la fecha y nuestra función `main_day_weeker`, podemos determinar exactamente el numero de semana en que se festejarán estas fechas.
En este caso no hay dudas de esas fechas se festejarán el día en que caen.

In [10]:
def sabado_santo(year): 
    """
    Function for calculating Sabado santo (Samana Santa's Saturday).
    """
    return gauss_easter(year) - timedelta(days = 1)

def easter_saturday(year):
    """
    Function for calculation easter's saturday.
    """
    return gauss_easter(year) + timedelta(days = 6)

def new_year(current_year):
    """
    January first calculation.
    """
    return datetime(current_year,1,1)

def christmas(current_year):
    """
    christmas calculation.
    """
    return datetime(current_year,12,25)



**CASO 2:** _Deterministas Irregulares_

No tan directo como las deterministas regulares, ya que estas fechas no se festejan el día en el caen, sino en un lunes particular en su mes. En este caso debemos de calcular el `datetime` cuando de festejará y luego usar `main_day_weeker` para saber el numero de semana fractional en que se festejará.

In [ ]:
def constitution_day(current_year):
    """
    First Monday of each February.
    """
    count = 0 
    for day in range(1,29):
        date = datetime(current_year,2,day)
        if date.weekday() == 0:
            count += 1
            if count == 1:
                return date
            
def benito_juarez_birthday(current_year):
    """
    Third Monday of each March.
    """
    count = 0
    for day in range(1, 32):  
        date = datetime(current_year, 3, day)
        if date.weekday() == 0:  
            count += 1
            if count == 3:
                return date

def mexican_revolution_day(current_year):
    """
    Since 2006 mexican government decreed day of the revolution will celebrated on third monday of november in each year.
     So, this function calculates when is that particular monday.
        """
    count = 0
    for day in range(1, 31):  
        date = datetime(current_year, 11, day)
        if date.weekday() == 0:
            count += 1
            if count == 3:
                return date
            
def father_day(current_year):
    """
    Third Sunday of each june.
    """
    count = 0
    for day in range(1, 31):  # june has 30 days
        date = datetime(current_year, 6, day)
        if date.weekday() == 6:  # sunday is equal to 6
            count += 1
            if count == 3:
                return date

def thanksgiving(current_year):
    """
    Fourth thursday of each november
    """
    count = 0
    for day in range(1, 31):  # november has 30 days
        date = datetime(current_year, 11, day)
        if date.weekday() == 3:  # monday is equal to 0
            count += 1
            if count == 4:
                return date

**CASO 3:** _No Deterministas_

Son fechas importantes que bien pueden ser puentes, sin embargo no existe una regla de correspondencia clara para definir cuando se festejaran exactamente. Usaremos el correspondiente `datetime` como argumento de `main_day_weeker` calculando tanto el número de semana en que se debería de festejar esta fecha como el de la semana anterior. Esto último nos asegura calcular la semana en que se festejará de manera inequivoca.

Ejemplo:
- Si una fracción inicia en jueves, el día festivo es en viernes y además la gente decide festejarlo el siguiente lunes. En este caso no nos afectaría pues la semana fractional es de jueves a miercoles.
- Si una fracción inicia semana en martes, el dia festivo es miercoles y además la gente decide fetejarlo el lunes anterior. En este caso tambien estamos cubiertos, pues hemos calculado el numero de la semana anterio.



In [12]:
def valentines_day(current_year):
    """
    Valentine's Day calculation
    """
    return datetime(current_year,2,14)

def mothers_day(current_day):
    """
    Mother's Day Calculation
    """
    return datetime(current_day,5,10)
def work_day(current_year):
    """
    Sometimes it could be a "puente" only if it is monday or friday.
    """
    date = datetime(current_year,5,1)
    return date

def independence_day(current_year):
    """
    Sometimes it could be a "puente" only if it is monday or friday.
    """
    date = datetime(current_year,9,16)
    return date

### Función `holly_weeks`
_Cálculo de semanas que mencionamos anteriormente._

In [13]:
def holly_weeks(current_year, weekday_calendar_starts):
    """
    Some weeks have special dates which no one want to miss them. 
    Those dates could be deterministic or probabilistic.
    """

    def deterministic_holly_weeks(current_year,weekday_calendar_starts):
        """
        Deterministic hollydays are those which have an specific rule to determinate them,
        for example mexican revolution day is third monday of each november, so this funcion return us 
        the list of those weeks which have these hollydays.
        """
        newyear = new_year(current_year)
        constitution = constitution_day(current_year)
        benito = benito_juarez_birthday(current_year)
        revolution = mexican_revolution_day(current_year)
        easter = easter_saturday(current_year)
        semana_santa = sabado_santo(current_year)
        christ = christmas(current_year)

        special_dates = [newyear,constitution,benito,revolution,easter,semana_santa,christ]
        calendar = main_day_weeker(current_year, weekday_calendar_starts)
        week_index = []

        for i in special_dates:
            week = calendar[i]
            week_index.append(week)

        return week_index

    def probabilistic_holly_weeks(current_year,weekday_calendar_starts):
        """
        Others dates don't let us get sure about whether the week which contains the date will the week when the date will celebrated.
        For example, figure out independence day takes on tuesday and owr fractional week begins also in tusday but people wants to celecrate in previous momday.
        It's worth to say, according the earlier case, if we take the weeks which have these dates and we take the previous week, we cover all the cases.
        So, you can intuit what this function does.
        """
    
        valentines = valentines_day(current_year)
        mom = mothers_day(current_year)
        work = work_day(current_year)
        independence = independence_day(current_year)

        special_dates = [valentines,mom,work,independence]
        calendar = main_day_weeker(current_year, weekday_calendar_starts)
        week_index = []

        for i in special_dates:
            week = calendar[i]
            week_index.append(week)
        
        before_week_index = []
        for k in week_index:
            before_week_index.append([k[0] - 1])

        return week_index + before_week_index

    regular = deterministic_holly_weeks(current_year, weekday_calendar_starts)
    irregular = probabilistic_holly_weeks(current_year, weekday_calendar_starts)

    gold = []                       # This block is looking for clean the list up.
    for i in regular + irregular:
        if i not in gold:
            gold.append(i)
    gold_num = [k[0] for k in gold]
    gold_num.sort()
    gold = [[k] for k in gold_num]

    return gold

Veamos una demostración de `holly_weeks`, calculando el número de semana para todas las fechas festivas:

*_Ejemplo para una fracción que inicia semana en dia particular_

In [14]:
print(f'{"AÑO"} - {"SEMANAS RESERVADAS POR FESTIVIDAD"}')
for year in range(2026,2051):
    reserved_weeks = holly_weeks(year,3)
    weeks = []
    for i in reserved_weeks:
        weeks.append(i[0])
    print(f'{year}-{weeks}')

AÑO - SEMANAS RESERVADAS POR FESTIVIDAD
2026-[0, 4, 5, 6, 10, 13, 14, 16, 17, 18, 35, 36, 45, 51]
2027-[0, 4, 5, 6, 10, 12, 13, 16, 17, 18, 36, 37, 45, 51]
2028-[0, 5, 6, 11, 15, 16, 17, 18, 36, 37, 46, 51]
2029-[0, 5, 6, 11, 13, 14, 16, 17, 18, 19, 36, 37, 46, 51]
2030-[0, 5, 6, 7, 11, 16, 17, 18, 19, 36, 37, 46, 51]
2031-[0, 5, 6, 7, 11, 15, 16, 17, 18, 19, 36, 37, 46, 52]
2032-[0, 4, 5, 6, 10, 12, 13, 16, 17, 18, 36, 37, 45, 51]
2033-[0, 5, 6, 11, 15, 16, 17, 18, 36, 37, 46, 51]
2034-[0, 5, 6, 11, 14, 15, 16, 17, 18, 36, 37, 46, 51]
2035-[0, 5, 6, 11, 12, 13, 16, 17, 18, 19, 36, 37, 46, 51]
2036-[0, 5, 6, 7, 11, 15, 16, 17, 18, 19, 36, 37, 46, 52]
2037-[0, 4, 5, 6, 10, 13, 14, 16, 17, 18, 35, 36, 45, 51]
2038-[0, 4, 5, 6, 10, 16, 17, 18, 36, 37, 45, 51]
2039-[0, 5, 6, 11, 14, 15, 16, 17, 18, 36, 37, 46, 51]
2040-[0, 5, 6, 11, 13, 14, 16, 17, 18, 19, 36, 37, 46, 51]
2041-[0, 5, 6, 7, 11, 16, 17, 18, 19, 36, 37, 46, 51]
2042-[0, 5, 6, 7, 11, 14, 15, 17, 18, 19, 36, 37, 46, 52]
2043-[0

## Función `maintenance_weeks_list`
_Imagina que puedes tener diferentes conjuntos de semanas de mantenimiento mutuamente excluyentes, asignando un identificador por unidad. Eso es lo que `maintenance_weeks_list` y `maintenance_weeks_path`_ hacen en conjunto al cambiar el parametro `maintenance_path`

In [16]:
def maintenance_weeks_list(current_year, weekday_calendar_starts, maintenance_path):
    """
    Select week indices for maintenance based on a path and the year characteristics.
    """
    weeks_per_fraction = weeks_expected_per_year // fractions_quantity
    reserved_weeks = weeks_expected_per_year - fractions_quantity * weeks_per_fraction
    
    def maintenance_weeks_paths(current_year, weekday_calendar_starts,reserved_weeks):
        """
        This function crafts a dictionarie with no hollyweeks in tis keys (datetimes),
        and also it bounds the dictionarie particulary.
        """
        if extra_week_indicator(current_year,weekday_calendar_starts):
            reserved_weeks +=1

        calendar = main_day_weeker(current_year, weekday_calendar_starts)
        gold = holly_weeks(current_year, weekday_calendar_starts)

        regular = {k:v for (k,v) in calendar.items() if v not in gold}
        list = [[i//7] for i in range(len(regular.values()))]
        regular = dict(zip(regular.keys(),list))
        bound = len(regular.keys()) // 7
        max_regular_len = bound // reserved_weeks * reserved_weeks

        dic = {k: v for k, v in regular.items() if v[0] < max_regular_len}

        return {k:[(v[0] + (current_year % fractions_quantity)) % (max_regular_len // reserved_weeks)] for (k,v) in dic.items()} # Esto no lo vuelve aleatorio                                                                                                                               # Pero si sería complicado calcular rápi
    maintenance_deserved_weeks = maintenance_weeks_paths(current_year, weekday_calendar_starts,reserved_weeks)                   
    lenght = len(maintenance_deserved_weeks.values()) // 7 // reserved_weeks
    matching_keys = [k for (k,v) in maintenance_deserved_weeks.items() if v[0] == maintenance_path % lenght]

    calendar = main_day_weeker(current_year,weekday_calendar_starts)

    dirty_list = []
    for i in matching_keys:
        dirty_list.append(calendar[i])

    maintenance_weeks = []
    for r in dirty_list:
        if r not in maintenance_weeks:
            maintenance_weeks.append(r)
        
  return maintenance_weeks

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 43)